# Importing data
Let's start with importing our packages. We will use scanpy, scvelo and cell bender. We will run this with a GPU to boost efficiency.

In [ ]:
!pip install git+https://github.com/broadinstitute/CellBender.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/broadinstitute/CellBender.git to /tmp/pip-req-build-etq_j38j
  Running command git clone -q https://github.com/broadinstitute/CellBender.git /tmp/pip-req-build-etq_j38j
     |████████████████████████████████| 96 kB 4.1 MB/s 
     |████████████████████████████████| 722 kB 60.1 MB/s 
     |████████████████████████████████| 3.2 MB 58.3 MB/s 
     |████████████████████████████████| 2.8 MB 48.3 MB/s 
     |████████████████████████████████| 84 kB 4.1 MB/s 
     |████████████████████████████████| 121 kB 73.6 MB/s 
     |████████████████████████████████| 90 kB 11.0 MB/s 
     |████████████████████████████████| 1.1 MB 66.4 MB/s 
     |████████████████████████████████| 100 kB 11.7 MB/s 
     |████████████████████████████████| 133 kB 74.1 MB/s 
  Created wheel for cellbender: filename=cellbender-0.2.0-py3-none-any.whl size=65997 sha256=446b52a859390e45eaa82e26233b07bb378

In [ ]:
# install packages

import os, sys
from google.colab import drive
drive.mount('/content/drive')

!pip install --quiet scvi-colab
from scvi_colab import install
install()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import anndata
import scvi
import scanpy as sc

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     |████████████████████████████████| 235 kB 9.1 MB/s 
     |████████████████████████████████| 51 kB 2.9 MB/s 
INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


FileNotFoundError: ignored

# Running Cellbender
Running cellBender on following samples. Requires GPU given --cuda argument.

In [ ]:
#sampleRawH5Matricies = ["/content/drive/MyDrive/SusztakLabFiles/scRNAseq/RawData/HK2770_SC/raw_feature_bc_matrix.h5", 
                        "/content/drive/MyDrive/SusztakLabFiles/scRNAseq/RawData/HK2833_SN/raw_feature_bc_matrix.h5", 
                        "/content/drive/MyDrive/SusztakLabFiles/scRNAseq/RawData/HK2896_SN/raw_feature_bc_matrix.h5"]
sampleRawH5Matricies = ["/content/drive/MyDrive/SusztakLabFiles/multiOme/RawData/HK2725_SN/raw_feature_bc_matrix.h5"]
#sampleNames = ["HK2770_SC", "HK2833_SN", "HK2896_SN"]
sampleNames = ["HK2725"]

for index, item in enumerate(sampleRawH5Matricies):
    outFile="/content/drive/MyDrive/SusztakLabFiles/scRNAseq/cellBenderOutput/"+sampleNames[index]+"_output.h5"
    !cellbender  remove-background \
        --input {item} \
        --output {outFile} \
        --expected-cells 10000 \
        --total-droplets-included 50000 \
        --fpr 0.01 \
        --cuda \
        --epochs 150


cellbender:remove-background: Command:
cellbender remove-background --input /content/drive/MyDrive/SusztakLabFiles/scRNAseq/RawData/HK2770_SC/raw_feature_bc_matrix.h5 --output /content/drive/MyDrive/SusztakLabFiles/scRNAseq/cellBenderOutput/HK2770_SC_output.h5 --expected-cells 10000 --total-droplets-included 50000 --fpr 0.01 --cuda --epochs 150
cellbender:remove-background: 2022-09-20 13:46:30
cellbender:remove-background: Running remove-background
cellbender:remove-background: Loading data from file /content/drive/MyDrive/SusztakLabFiles/scRNAseq/RawData/HK2770_SC/raw_feature_bc_matrix.h5
cellbender:remove-background: CellRanger v3 format
cellbender:remove-background: Trimming dataset for inference.
cellbender:remove-background: Including 27081 genes that have nonzero counts.
cellbender:remove-background: Prior on counts in empty droplets is 17
cellbender:remove-background: Prior on counts for cells is 4828
cellbender:remove-background: Excluding barcodes with counts below 15
cellbend

# Convert to Anndata object
Now, let's convert this into a anndata object. We will then save it as a .h5ad for importing into scanpy or seurat.

In [ ]:
for index, item in enumerate(sampleRawH5Matricies):
    outFile="/content/drive/MyDrive/SusztakLabFiles/scRNAseq/cellBenderOutput/"+sampleNames[index]+"_output_filtered.h5"
    adata = sc.read_10x_h5(outFile, genome="hg19")
    adata.write("/content/drive/MyDrive/SusztakLabFiles/scRNAseq/cellBenderOutput/"+sampleNames[index]+'_cellBender.h5ad')
    print(adata)